<a href="https://colab.research.google.com/github/sk-gr/ml/blob/main/Refactoring_Python_DNN_with_core_torch_nn_components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
lab_idx = 1

if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False  # change to True re-run setup

!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab01
notebooks/  text_recognizer/


In [ ]:
from pathlib import Path
import requests


def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/main/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename


data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

In [ ]:
import gzip
import pickle


def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)

In [ ]:
import torch


x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

print(x_train, y_train, sep="\n")

y_train[0], x_train[0, ::2]

x_train.ndim, y_train.ndim

n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
torch.Size([50000])


<ipython-input-20-93ee5ed35295>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train, y_train, x_valid, y_valid = map(


In [66]:
# re-execute this cell for more samples
import random

import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes

import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(*metadata.DIMS)).image  # the image itself

tensor(2)


In [ ]:
import math
import torch


weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

bs = 64  # batch size

xb = x_train[0:bs]  # a batch of inputs
outs = model(xb)  # outputs on that batch

print(outs[0], outs.shape)  # outputs on the first element of the batch

tensor([-1.9845, -2.2457, -2.2202, -2.9017, -2.4166, -2.3481, -2.3291, -2.3609,
        -1.9366, -2.6244], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [ ]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.0469)


In [ ]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [ ]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [ ]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3562, grad_fn=<NegBackward0>) tensor(2.3026)


In [ ]:
loss = loss_func(outs, yb)

loss.backward()

In [ ]:
bias.grad

tensor([ 0.0320, -0.0325,  0.0466, -0.0593, -0.0263,  0.0282,  0.0103,  0.0199,
         0.0320, -0.0509])

In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0827, grad_fn=<NegBackward0>) tensor(1.)


In [ ]:
lr = 0.5  # learning rate hyperparameter
epochs = 2  # how many epochs to train for

for epoch in range(epochs):  # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):  # in batches of size bs, so roughly n / bs of them
        start_idx = ii * bs  # we are ii batches in, each of size bs
        end_idx = start_idx + bs  # and we want the next bs entires

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad():  # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # ACHTUNG: PyTorch doesn't assume you're done with gradients
            #          until you say so -- by explicitly "deleting" them,
            #          i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()

In [24]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [25]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))  # should be unchanged from above!

tensor(0.0827, grad_fn=<NllLossBackward0>) tensor(1.)


In [26]:
from torch import nn


class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()  # the nn.Module.__init__ method does import setup, so this is mandatory
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [27]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()  # instantiated as an object
print(model(xb)[:4])  # callable like a function
loss = loss_func(model(xb), yb)  # composable like a function
loss.backward()  # we can still take gradients through it
print(model.weights.grad[::17,::2])  # and they show up in the .grad attribute

tensor([[ 0.1781, -0.5651, -0.0313,  0.1976, -0.0446, -0.3763,  0.5980, -0.5332,
         -0.3749, -0.1366],
        [ 0.2445, -0.5270,  0.1161, -0.1365,  0.4139,  0.0527,  0.6680,  0.2940,
         -0.9837, -0.0812],
        [ 0.1678, -0.3692, -0.0647, -0.1343,  0.1190, -0.4345,  0.6747,  0.1757,
          0.0255,  0.0274],
        [-0.0444, -0.8932, -0.2095,  0.3465,  0.0835,  0.1170,  0.4541,  0.5866,
         -0.5395, -0.0260]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 9.1295e-03,  7.6317e-03,  1.0138e-02,  1.0776e-02,  3.1128e-03],
        [ 0.

In [28]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0227, -0.0234,  0.0283,  ..., -0.0156, -0.0457,  0.0340],
        [ 0.0055, -0.0035,  0.0368,  ...,  0.0205, -0.0277, -0.0223],
        [-0.0024, -0.0102,  0.0545,  ..., -0.0282, -0.0437,  0.0199],
        ...,
        [-0.0358, -0.0084,  0.0212,  ..., -0.0338, -0.0056,  0.0136],
        [-0.0032, -0.0090,  0.0117,  ..., -0.0075,  0.0010, -0.0719],
        [-0.0183,  0.0083, -0.0028,  ...,  0.0281,  0.0451, -0.0180]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [29]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs
            end_idx = start_idx + bs
            xb = x_train[start_idx:end_idx]
            yb = y_train[start_idx:end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():  # finds params automatically
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [30]:
print(accuracy(model(xb), yb))

tensor(1.)


In [31]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [32]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)  # pytorch finds the nn.Parameters inside this nn.Module

    def forward(self, xb):
        return self.lin(xb)  # call nn.Linear.forward here

In [33]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))  # loss is still close to 2.3

tensor(2.2346, grad_fn=<NllLossBackward0>)


In [34]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [35]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0060,  0.0284,  0.0328,  ...,  0.0236,  0.0084, -0.0055],
        [ 0.0224,  0.0170, -0.0151,  ...,  0.0126, -0.0323,  0.0296],
        [-0.0242, -0.0030, -0.0137,  ...,  0.0073, -0.0313,  0.0039],
        ...,
        [-0.0133,  0.0056,  0.0116,  ..., -0.0208,  0.0299,  0.0163],
        [ 0.0222, -0.0321,  0.0272,  ..., -0.0008, -0.0167,  0.0114],
        [ 0.0349,  0.0239, -0.0048,  ..., -0.0186,  0.0119,  0.0233]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0038,  0.0309,  0.0311, -0.0297,  0.0222,  0.0321,  0.0075, -0.0030,
         0.0050,  0.0319], requires_grad=True)


In [36]:
from torch import optim


def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [37]:
model = MNISTLogistic()
opt = configure_optimizer(model)

print("before training:", loss_func(model(xb), yb), sep="\n\t")

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training:", loss_func(model(xb), yb), sep="\n\t")

before training:
	tensor(2.4524, grad_fn=<NllLossBackward0>)
after training:
	tensor(0.8417, grad_fn=<NllLossBackward0>)


In [40]:
from text_recognizer.data.util import BaseDataset


train_ds = BaseDataset(x_train, y_train)

In [41]:
BaseDataset??

In [42]:
model = MNISTLogistic()
opt = configure_optimizer(model)


for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]  # xb and yb in one line!
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.8639, grad_fn=<NllLossBackward0>)


In [43]:
from torch.utils.data import DataLoader


train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [44]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit

In [45]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.8492, grad_fn=<NllLossBackward0>)


In [46]:
from text_recognizer.models.mlp import MLP


MLP.fit = fit  # attach our fitting loop

In [47]:
MLP.forward??

In [48]:
MLP.__init__??

In [49]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [50]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [51]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0168, -0.0066,  0.0221,  ...,  0.0214, -0.0126,  0.0088],
        [ 0.0347, -0.0178, -0.0268,  ..., -0.0236,  0.0246, -0.0100],
        [ 0.0181, -0.0151,  0.0192,  ..., -0.0234,  0.0252,  0.0079],
        ...,
        [ 0.0244, -0.0243, -0.0292,  ...,  0.0260,  0.0354, -0.0351],
        [-0.0297, -0.0184, -0.0120,  ...,  0.0073, -0.0075,  0.0336],
        [ 0.0029,  0.0281,  0.0158,  ..., -0.0302,  0.0251,  0.0205]],
       requires_grad=True)

In [52]:
%%time

print("before training:", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training:", loss_func(model(xb), yb))

before training: tensor(2.3279, grad_fn=<NllLossBackward0>)
after training: tensor(0.3664, grad_fn=<NllLossBackward0>)
CPU times: user 12.9 s, sys: 4.84 ms, total: 12.9 s
Wall time: 13.1 s


In [53]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

☹️


In [54]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

tensor(0.2467, grad_fn=<NllLossBackward0>)

In [55]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [56]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.2300, grad_fn=<NllLossBackward0>)
CPU times: user 13.1 s, sys: 8.76 ms, total: 13.1 s
Wall time: 13.3 s


In [62]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open("wb").write(content)

    def setup(self):
        with gzip.open(self.path, "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

        x_train, y_train, x_valid, y_valid = map(
            torch.tensor, (x_train, y_train, x_valid, y_valid)
            )

        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=2 * self.bs, shuffle=False)

In [63]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.val_dataloader()

    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

    print("before start of training:", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()
    for epoch in range(epochs):
        self.train()
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

        print(epoch, valid_loss / len(val_dataloader))


MNISTLogistic.fit = fit
MLP.fit = fit

In [65]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

FileNotFoundError: ignored